In [1]:
import numpy as np, itertools as itr

In [2]:
import ase, ase.io

In [3]:
bvs_params = {
    'Li':{
        'R0': 1.1745,
        'b' : 0.514,
    },
    'Na':{
        'R0':1.5766,
        'b': 0.475,
    },
}

In [4]:
alkali = 'Na'

In [5]:
cif = 'NYS_base_file.cif'
#atoms = ase.io.read(cif)

In [6]:
import mendeleev

In [7]:
class Ions:
    
    def __init__(self,atoms,ionLabel):
        self.atomicNumber = mendeleev.element(ionLabel).atomic_number
        self.atoms = atoms[atoms.numbers==self.atomicNumber]
        self.scaledPositions = self.atoms.get_scaled_positions().copy()
        self.unscaledPositions = np.inner(self.scaledPositions,self.atoms.cell.T)
        
    def get_ionsWithinCutoff(self,position,cutoff):
        lowerBoundary = position - cutoff
        upperBoundary = position + cutoff
        withinBoundary = np.all(((self.unscaledPositions < upperBoundary) & (self.unscaledPositions > lowerBoundary)),axis=1)
        return self.unscaledPositions[withinBoundary] # counter ions
    

In [8]:
class UnitCell:
    def __init__(self,cif):
        self.atoms = ase.io.read(cif)
        self.latticeParameters = np.linalg.norm(self.atoms.cell,axis=1)
        self.latticeVectors = self.atoms.cell
    def get_ionOfType(self,ionLabel):
        ions = Ions(atoms = self.atoms, ionLabel=ionLabel)
        return ions

In [9]:
unitCell = UnitCell(cif)

mainIons = unitCell.get_ionOfType(alkali)
counterIons = unitCell.get_ionOfType('O')

In [10]:
dx,dy,dz = 0.4,0.4,0.4
resolution = np.array([dx,dy,dz])

In [11]:
sizeOfValenceMismatchArray = tuple((unitCell.latticeParameters / resolution).astype(int))
valenceMismatch = np.ones(sizeOfValenceMismatchArray)

In [12]:
def include_atomsFrom_nns_unitCells(scaledPositions):
    nnsCoordinates = [np.array(i) for i in itr.product([-1,0,1],repeat=3)]
    totalAtomsIncludingNNS = np.concatenate([scaledPositions + nnsCoordinate for nnsCoordinate in nnsCoordinates])
    return totalAtomsIncludingNNS

In [13]:
counterIons.scaledPositions = include_atomsFrom_nns_unitCells( counterIons.scaledPositions )

In [14]:
counterIons.unscaledPositions = np.inner(counterIons.scaledPositions,unitCell.latticeVectors.T)

In [15]:
from numpy import ravel_multi_index as rav

In [16]:
cutoffRadii = 6.

In [17]:
R0 = bvs_params[alkali]['R0']
b = bvs_params[alkali]['b']

In [18]:
numberOfVoxels = len(valenceMismatch.flatten())

for voxelFlatIndex in range(numberOfVoxels):
    voxelTupleIndex = np.unravel_index(voxelFlatIndex,valenceMismatch.shape)
    # I don't understand this
    voxelScaledPosition = (2*np.array(voxelTupleIndex)+1.)/(2*np.array([valenceMismatch.shape]).astype(float))[0]
    voxelUnscaledPosition = np.dot(unitCell.latticeVectors.T,voxelScaledPosition)
    
    counterIonsWithinCutoff = counterIons.get_ionsWithinCutoff(position=voxelUnscaledPosition,cutoff=cutoffRadii)
   
    distanceToCounterIons = np.linalg.norm(counterIonsWithinCutoff-voxelUnscaledPosition,axis=1)
    Ri = distanceToCounterIons
    Ri = Ri[Ri<cutoffRadii]
    
    valenceMismatch[voxelTupleIndex] = np.abs( np.sum( np.exp((R0-Ri)/b) ) - 1 )
    

In [20]:
output_filename = './grds/NYS_Mismatch.grd'

#output_filename = cif.split('.')[0] + '.grd'
with open(output_filename,"w") as savefile:
    savefile.write("Bond Valence Sum Difference\r") # Title
    from ase.geometry import cell_to_cellpar
    cellParams = cell_to_cellpar(unitCell.latticeVectors) # get ABC alpha, beta, gamma
    savefile.write(" ".join([str(k) for k in cellParams])+"\r" )
    savefile.write(" ".join([str(k) for k in valenceMismatch.shape])+"\r" )
    for i in np.nditer(valenceMismatch.flatten()):
        savefile.write("%.6f  "%(i)) # Write each valence difference value